In [ ]:
import flexlib
from matplotlib import pyplot as plt
import time
from IPython.display import display, clear_output
import threading
%matplotlib inline
import importlib
importlib.reload(flexlib)

## Initialize session

In [ ]:
# create a motorController object with one stepper connected to the first position with 2 whiskers that will whisk from 20 degrees to 120 degrees
# when the whisker signal gets 100 steps above its value read during init(), it will palpate 3 times pushing 22 steps past detected contact position,
# then pull back to 6 steps before contact position
ctrl = flexlib.MotorController(exp_tag='my_exp_title', shield_addrs=[0],  motor_startup='1', num_whiskers_per_col=2, \
                               prot_steps=flexlib.deg_to_steps(120), ret_steps=flexlib.deg_to_steps(20), contact_thresh=100,\
                              push_steps=22, pull_steps=6, palpate_num=3, contact_behavior='palpate') 

In [ ]:
# initialize the experiment - this will complete the setup process, home the motors, and start data streaming
ctrl.init()     

## Run experiment
Whisks constantly until stop_whisking() is called. If contact_behavior='palpate', it will palpate palpate_num times when a whisker signal 
is contact_thresh steps above resting. This is handled in a thread that is also reading in data, so other commands could still be run 
while whisking. See the bottom of this notebook for an example that spawns a thread to plot the incoming whisker signals.

In [ ]:
ctrl.start_whisking()

In [ ]:
ctrl.stop_whisking()

In [ ]:
# shutdown experiment and save data log (it is reccommended to restart the kernel before starting a new experiment)
try:
    stop_animation()
except:
    pass
ctrl.end_exp()

#### Some simple examples for other available commands

In [ ]:
# example for setting the whisk speed and acceleration
# default is 1500 speed (move as quickly as the arduino can loop) and 500 accel
# this must be run after ctrl.init()
ctrl.set_speed(1500)
ctrl.set_accel(600)

In [ ]:
# example for setting a new target protraction and retraction for all columns, values are relative to homed 0 value
prot_steps = flexlib.deg_to_steps(120)
ret_steps = flexlib.deg_to_steps(20)
ctrl.default_prot_steps = prot_steps
ctrl.default_ret_steps = ret_steps
ctrl.set_prot(prot_steps)
ctrl.set_ret(ret_steps)

In [ ]:
# change contact threshold
new_thresh = 75

# stop whisking and let oscillations stop so a new rest val can be read
ctrl.stop_whisking()
ctrl.whisking_stopped.wait()
time.sleep(2)
# read rest val and set new threshold
for i in range(ctrl.num_whiskers):
    rest_val = ctrl.last_vals[i]
    ctrl.thresh_arr[i] = rest_val + new_thresh

In [ ]:
# manually perform 3 whisks (does not check for contacts)
for _ in range(3):
    ctrl.whisk()
    ctrl.whisking_stopped.wait()

In [ ]:
# whisk 5 times while checking for contacts
whisk_count = 1
target_count = 5
ctrl.start_whisking()
ctrl.data_ready.wait()
while whisk_count < target_count:
    ctrl.whisking_stopped.wait()
    ctrl.data_ready.wait()
    if not ctrl.is_palpating:
        whisk_count += 1
    while ctrl.is_palpating:
        time.sleep(.1)
ctrl.stop_whisking()

In [ ]:
# disable palpating behavior
ctrl.contact_behavior = None

In [ ]:
# enable palpating behavior
ctrl.contact_behavior = 'palpate'

In [ ]:
# print the time in microseconds the last 10 samples were taken and print values and positions for each whisker at those times
print(ctrl.timestamps[-11:-1])
for i in range(ctrl.num_whiskers):
    print('whisker %d'%i)
    print(ctrl.whisker_vals[i][-11:-1])
    print(ctrl.whisker_steps[i][-11:-1])

In [ ]:
# view the data log (pandas dataframe)
ctrl.log

In [ ]:
# save the log to a .csv file
ctrl.write_log()

### Disply signals
Create a thread that will display a live feed of the last 300 samples for each whisker

In [ ]:
# number of samples to display
disp_samples = 300
# number of seconds between updates
update_interval = 0.25

# initialize variables
signal_data = [[] for _ in range(ctrl.num_whiskers)]  
stop_flag = False

# Set up the plot
fig, ax = plt.subplots()
lines = [ax.plot([], [], lw=2)[0] for _ in range(ctrl.num_whiskers)] 
ax.set_xlim(0, disp_samples)
ax.set_ylim(-0.2, 1.2)
ax.set_title('Live Signal Data')
ax.set_xlabel('Sample')
ax.set_ylabel('Volts')

# Function to manually update the plot
def update_plot():
    global ctrl
    global disp_samples
    global update_interval
    while not stop_flag:
        for i in range(ctrl.num_whiskers):  # Loop through each signal
            if len(ctrl.whisker_vals[i]) < disp_samples:
                signal_data[i] = ctrl.whisker_vals[i]
            else:
                signal_data[i] = ctrl.whisker_vals[i][-301:-1]

            signal_data[i] = [j * (1.1 / 1024) for j in signal_data[i]]
            x_data = list(range(len(signal_data[i])))
            y_data = signal_data[i]
            lines[i].set_data(x_data, y_data)
        
        ax.set_xlim(max(0, len(signal_data[0]) - disp_samples), len(signal_data[0]))  # Update x-axis limit dynamically
        clear_output(wait=True)
        display(fig)
        time.sleep(update_interval)  # Adjust the sleep time to control update rate

# Start the plot update function in a separate thread
plot_thread = threading.Thread(target=update_plot)
plot_thread.daemon = True
plot_thread.start()

# Function to stop the data acquisition and animation
def stop_animation():
    global stop_flag
    stop_flag = True

In [ ]:
# stop animation
stop_animation()